In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import csv
import nltk
from matplotlib import pyplot as plt
import os
import re
import warnings
import random
warnings.filterwarnings('ignore')
from collections import Counter,defaultdict
from nltk import word_tokenize
from math import log
import string
import pickle
from math import log
import gensim
from gensim import corpora
from gensim import models
from string import digits
from collections import defaultdict
from scipy.spatial.distance import cosine
import ast
cwd = os.getcwd()

In [2]:
osname = os.name
if osname =='Windows':
    sym = "//"
else:
    sym = "/"

In [3]:
r_path=cwd+sym+'nips_reviewer_data'+sym+'reviewers.txt'
review_df=pd.read_csv(r_path,sep='\t',header=None)
review_df.columns=['sno','name']
review_lt=list(review_df.name)
review_sno={val:ind for ind,val in enumerate(review_lt)}

## Loading Necessary files

In [98]:
path1=cwd+sym+'Saved_Items'+sym
with open(path1+'dictionary.pkl','rb') as f:
    dictionary=pickle.load(f)
with open(path1+'tfidf.pkl','rb') as f:
    tfidf=pickle.load(f)
with open(path1+'ldaModel250','rb') as f:
    lda_model=pickle.load(f)
with open(path1+'cocit.pkl','rb') as f:
    cocit_mat=pickle.load(f)
with open(path1+'bibcoupling.pkl','rb') as f:
    bc_mat=pickle.load(f)
with open(path1+'distance.pkl','rb') as f:
    dist=pickle.load(f)
with open(path1+'paper_vec2.pkl','rb') as f:
    paper_vec2=pickle.load(f)
with open(path1+'paper_id.pkl','rb') as f:
    paper_id=pickle.load(f)
with open(path1+'reviewer_vec.pkl','rb') as f:
    reviewer_vec=pickle.load(f)
with open(path1+'topic_community.pkl','rb') as f:
    topic_comm=pickle.load(f)
with open(path1+'paper_auth.pkl','rb') as f:
    paper_auth=pickle.load(f)
with open(path1+'reviewer_matrix.pkl','rb') as f:
    reviewer_matrix=pickle.load(f)
with open(path1+'reviewer_vec_bow.pkl','rb') as f:
    reviewer_vec_bow=pickle.load(f)
reviewer_detail=pd.read_pickle(path1+'auth_detail.pkl')

In [115]:
with open(path1+'reviewer_matrix_inter_baseline.pkl','rb') as f:
    reviewer_matrix=pickle.load(f)

## New Paper's Title and Abstract

In [5]:
title='Learning from Multiple Sources'
abstract='''
Categorization is a central activity of human cognition. When an individual is asked to categorize a sequence of items, context effects arise: categorization of one item influences category decisions for subsequent items. Specifically, when experimental subjects are shown an exemplar of some target category, the category prototype appears to be pulled toward the exemplar, and the prototypes of all nontarget categories appear to be pushed away. These push and pull effects diminish with experience, and likely reflect long-term learning of category boundaries. We propose and evaluate four principled probabilistic (Bayesian) accounts of context effects in categorization. In all four accounts, the probability of an exemplar given a category is encoded as a Gaussian density in feature space, and categorization involves computing category posteriors given an exemplar. The models differ in how the uncertainty distribution of category prototypes is represented (localist or distributed), and how it is updated following each experience (using a maximum likelihood gradient ascent, or a Kalman filter update). We find that the distributed maximum-likelihood model can explain the key experimental phenomena. Further, the model predicts other phenomena that were confirmed via reanalysis of the experimental data. Categorization is a central activity of human cognition. We continually make decisions about characteristics of objects and individuals: Is the fruit ripe? Does your friend seem unhappy? Is your car tire flat? Should this manuscript be accepted for publication in NIPS? When an individual is asked to categorize a sequence of items, context effects arise: categorization of one item influences category decisions for subsequent items. Intuitive naturalistic scenarios in which context effects occur are easy to imagine. For example, if one lifts a medium-weight object after lifting a light-weight or heavy-weight object, the medium weight feels heavier following the light weight than following the heavy weight. Although the object-contrast effect might be due to fatigue of sensory-motor systems, many context effects in categorization are purely cognitive and cannot easily be attributed to neural habituation. For example, if you are reviewing a set of conference papers, and the first three in the set are dreadful, then even a mediocre paper seems like it might be above threshold for acceptance. Another example of a category boundary shift due to context is the following. Suppose you move from San Diego to Pittsburgh and notice that your neighbors repeatedly describe muggy, somewhat overcast days as Ólovely.Ó Eventually, your notion of what constitutes a lovely day accommodates to your new surroundings. As we describe shortly, experimental studies have shown a fundamental link between context effects in categorization and long-term learning of category boundaries. We believe that context effects can be viewed as a reflection of a trial-to-trial learning, and the cumulative effect of these trial-to-trial modulations corresponds to what we classically consider to be category learning. Consequently, any compelling model of category learning should also be capable of explaining context effects. stimulus dimension ABCD example Figure 1: Schematic depiction of sequential effects in categorization 
'''
text=title+abstract

## PreProcessing

In [6]:
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
st = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer 
import re

def cleantext(text):
    
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    
    # remove punctuation
    table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    strip = text.translate(table)
    
    
    # Tokenizer
    tokens = word_tokenize(strip)
    
    # Convert into lower case 
    proc_text = [w.lower() for w in tokens]
    
    # Remove stopwords
    proc_text = [word for word in proc_text if word not in st]
    
    
    #Storing only Lemmmatized words
    lemmatizer=WordNetLemmatizer()
    lemma_text=[lemmatizer.lemmatize(word) for word in proc_text]

    return " ".join(lemma_text)


t = "i am groot2'7"    
cleantext(t)

'groot'

In [7]:
def get_column_slice(array_or_dataframe,coloumn):
    return np.asarray(array_or_dataframe)[:, coloumn]

In [8]:
num_topics=250

## Consolidated Functions 

In [9]:
def topic_level(text):
    # Loading saved Lda model,dictionary,tfidf vectorizer
    
    clean_text=list(word_tokenize(cleantext(text))) 
    bow_text=dictionary.doc2bow(clean_text)
    tfidf_text=tfidf[bow_text]
    topic=lda_model[tfidf_text]

    topic_vec=np.zeros(num_topics)
    for ind,wt in topic:
        topic_vec[ind]=wt
    
    # Topic of interest and community
    
    no_top_topic=5
    toi=[]#topic of interest
    if len(topic)<no_top_topic:
        for ind,wt in topic:
            toi.append(ind)
    else:
        topic.sort(key=lambda x:x[1],reverse=True)
        for i in range(no_top_topic):
            toi.append(topic[i][0])

    
    toi_comm=[topic_comm[i] for i in toi]
    toi_comm=set(toi_comm)
    topic_rest=[]
    for index,comm_no in enumerate(topic_comm):
        if comm_no in toi_comm and not index in toi:
            topic_rest.append(index)
    
    return topic_vec,toi,topic_rest

In [10]:
def paper_level(toi,topic_rest):
    poi=[]#papers having maximuim content of toi
    paper_vec_lt=list(paper_vec2.values())
    for i in toi:
        samp=get_column_slice(paper_vec_lt,i)
        ind=np.argmax(np.array(samp))
        poi.append(ind)
    puc=[]#papers under consideration
    for i in topic_rest:
        topic_paper=get_column_slice(paper_vec_lt,i)
        n1=len(np.where(topic_paper>0)[0])
        thres=sum(topic_paper)/n1
        ind_list=np.where(topic_paper>thres)[0]
        for ele in ind_list:
            puc.append(ele)
    puc=list(set(puc))
    
    paper_scr=[]
    for i in puc:
        num=bc_mat[i]+cocit_mat[i]
        den=min([dist[i][j] for j in poi])
        if den==0:
            scr=float('inf')
        else:
            scr=num/den
        paper_scr.append((scr,i))
    
    scr=[scr for scr,ind in paper_scr]
    paper_scr.sort(reverse=True)
    for i in range(300):
        poi.append(paper_scr[i][1])
    val=np.percentile(scr,75)# taking into acccount only top 25% entries
    potential_paper=[]
    for score,ind in paper_scr:
        if score>val:
            potential_paper.append(ind)
    
    
    return poi,potential_paper


In [110]:
def reviewer_pred(poi,potential_paper,topic_vec,text):
    roi=[]
    ct_roi=[]
    for i in poi:
        roi=roi+paper_auth[i]
    #print(len(set(roi)))
    rest_reviewer=[]
    for i in potential_paper:
        if not paper_auth[i] in roi:
            rest_reviewer=rest_reviewer+paper_auth[i]
    
    
    
    npaper=list(reviewer_detail['pc'])
    citn=list(reviewer_detail['cn'])
    
    clean_text=list(word_tokenize(cleantext(text))) 
    bow_text=dictionary.doc2bow(clean_text)
    tfidf_text=tfidf[bow_text]
    n=len(dictionary)

    bow_vec=np.zeros(n, dtype = float)
    for ind,wt in tfidf_text:
        bow_vec[ind]=wt    
    
    dec_scr=[]
    for i in roi:
        sim=1-cosine(reviewer_vec[i],topic_vec)
        bow_scr=1-cosine(reviewer_vec_bow[i],bow_vec)
        if np.isnan(sim):
            sim=0
        if np.isnan(bow_scr):
            bow_scr=0
        #val=0.1*npaper[i]/max(npaper)+0.1*citn[i]/max(citn)+0.8*sim
        val=0.6*sim+0.4*bow_scr
        #val2=npaper[i]/max(npaper)+citn[i]/max(citn)
        dec_scr.append((val,i))
    dec_scr.sort(reverse=True)
    start_node=dec_scr[0][1]
    
    n=len(review_lt)
    total_node=set([i for i in range(n)])
    invalid_node=total_node-set(roi)-set(rest_reviewer)
    
    custom_matrix=[]
    for i in range(n):
        temp=[]
        for j in range(n):
            temp.append(reviewer_matrix[i][j])
        custom_matrix.append(np.array(temp))
    for node in invalid_node:
        for i in range(n):
            custom_matrix[node][i]=0
            custom_matrix[i][node]=0
    col_sum=[sum(x) for x in zip(*custom_matrix)]

    transition_prob=[]
    for i in range(n):
        op=[]
        for j in range(n):
            if col_sum[j]:
                val=custom_matrix[i][j]/col_sum[j]
            else:
                val=0
            op.append(val)
        transition_prob.append(op)
        
    ini_state=[0]*n
    ini_state[start_node]=1
    len_op=10#No of items to be shown as output
    new_state=ini_state
    for i in range(len_op):
        new_state=np.dot(0.85,np.dot(transition_prob,new_state))+np.dot(0.15,ini_state)
        
    op=new_state.argsort()[-10:][::-1]
    #op=new_state.argsort()[::-1]
    res=[ele+1 for ele in op]
    
    return res
    

In [111]:
def predict(text):
    topic_vec,toi,topic_rest=topic_level(text)
    poi,potential_paper=paper_level(toi,topic_rest)
    pred=reviewer_pred(poi,potential_paper,topic_vec,text)
    
    return pred

## Evaluation

In [13]:
path=path1+'Testing'+sym+'trec_judgments.txt'
ordinal=[]
value=[]
judge={}
with open(path,'r',encoding="utf8")as f:
    for line in f:
        ordinal=list(line.split('\t'))
        value=[int(ordinal[2]),int(ordinal[3][:-1])]
        if int(ordinal[0][4:]) in judge:
            judge[int(ordinal[0][4:])].append(value)
        else :
            judge[int(ordinal[0][4:])]=[value]
            
path=path1+'Testing'+sym+'trec_judgments_additional.txt'
with open(path,'r',encoding="utf8")as f:
    for line in f:
        ordinal=list(line.split('\t'))
        value=[int(ordinal[2]),int(ordinal[3][:-1])]
        if int(ordinal[0]) in judge:
            judge[int(ordinal[0])].append(value)
        else :
            judge[int(ordinal[0])]=[value]


In [14]:
def precision_k(res,gnd_tht,k,paper_no):
    actual=gnd_tht[paper_no]
    actual_auth=[i[0] for i in actual]
    rating=[i[1] for i in actual]
    n=len(actual)
    soft_p=0
    for i in range(k):
        pred=res[i]
        for j in range(n):
            if actual_auth[j]==pred:
                if rating[j]==3:
                    soft_p+=1
                elif rating[j]==2:
                    soft_p+=0.67
                elif rating[j]==1:
                    soft_p+=0.33
                else:
                    break
    soft_p=soft_p/k
    return soft_p

In [17]:
test_abs=pd.read_pickle('test_abs')
n=len(test_abs)
ctr=0
avg_precision=[0]*11
valid_file=list(judge.keys())
for i in range(n):
    if i in valid_file:        
        res=predict(test_abs['abstract'].iloc[i])
        #print(res)
        for k in range(1,11):
            avg_precision[k]+=precision_k(res,judge,k,i)
        ctr+=1
avg_precision=[ele/ctr for ele in avg_precision]

In [18]:
avg_precision[1:]

[0.3332352941176471,
 0.396764705882353,
 0.316764705882353,
 0.2669852941176471,
 0.27041176470588235,
 0.2547549019607844,
 0.24357142857142863,
 0.23025735294117644,
 0.22098039215686271,
 0.2194705882352941]

## Handling Reviewers not present in Ground Thruth

In [42]:
def reviewer_404(res,ground_tht,paper_no):
    actual=ground_tht[paper_no]
    actual_auth=[i[0] for i in actual]
    rating=[i[1] for i in actual]
    n=len(actual)
    output=[]
    for auth in res:
        if auth not in actual_auth:
            sim_scr=[0]*4
            count=[0]*4
            vec1=reviewer_vec_bow[auth-1]
            for i in range(n):
                val=1-cosine(vec1,reviewer_vec_bow[actual_auth[i]-1])
                if np.isnan(val):
                    val=0
                sim_scr[rating[i]]+=val
                count[rating[i]]+=1
            sim_scr=[sim_scr[i]/count[i] for i in range(4) if count[i]]
            max_scr=max(sim_scr)
            ind=sim_scr.index(max_scr)
            if max_scr<0.
            output.append([auth,ind])
            #print(sim_scr)
    return output
                
    
    
    

In [33]:
res1=predict(test_abs['abstract'].iloc[0])
op=reviewer_404(res1,judge,0)

In [34]:
op

[[162, 3], [234, 2], [23, 3], [12, 0], [164, 0]]

In [112]:
def precision_k_404(res,gnd_tht,k,paper_no):
    actual=gnd_tht[paper_no]
    actual_auth=[i[0] for i in actual]
    rating=[i[1] for i in actual]
    n=len(actual)
    soft_p=0
    for i in range(k):
        pred=res[i]
        for j in range(n):
            if actual_auth[j]==pred:
                if rating[j]==3:
                    soft_p+=1
                elif rating[j]==2:
                    soft_p+=0.67
                elif rating[j]==1:
                    soft_p+=0.33
                else:
                    break
    rating_gen=reviewer_404(res,gnd_tht,paper_no)
    auth_gen=[i[0] for i in rating_gen]
    rating_gen=[i[1] for i in rating_gen]
    n1=len(auth_gen)
    for i in range(k):
        pred=res[i]
        for j in range(n1):
            if auth_gen[j]==pred:
                if rating_gen[j]==3:
                    soft_p+=1
                elif rating_gen[j]==2:
                    soft_p+=0.67
                elif rating_gen[j]==1:
                    soft_p+=0.33
                else:
                    break
    soft_p=soft_p/k
    return soft_p

In [118]:
test_abs=pd.read_pickle('test_abs')
n=len(test_abs)
ctr=0
avg_precision=[0]*11
valid_file=list(judge.keys())
for i in range(n):
    if i in valid_file:        
        res=predict(test_abs['abstract'].iloc[i])
        #print(res)
        for k in range(1,11):
            avg_precision[k]+=precision_k_404(res,judge,k,i)
        ctr+=1
avg_precision=[ele/ctr for ele in avg_precision]

In [119]:
avg_precision[1:]

[0.5291176470588236,
 0.5388235294117647,
 0.5618627450980394,
 0.5586764705882353,
 0.568529411764706,
 0.5619607843137255,
 0.5572689075630253,
 0.5538235294117647,
 0.55218954248366,
 0.5479117647058824]

### Insight about our results

In [20]:
def gndtht_stats(res,gnd_tht,paper_no):
    actual=gnd_tht[paper_no]
    actual_auth=[i[0] for i in actual]
    rating=[i[1] for i in actual]
    ctr_3=0
    ctr_2=0
    ctr_1=0
    ctr_0=0
    ctr_not=0    
    for ele in res:
        if ele in actual_auth:
            ind=actual_auth.index(ele)
            if rating[ind]==3:
                ctr_3+=1
            elif rating[ind]==2:
                ctr_2+=1
            elif rating[ind]==1:
                ctr_1+=1
            else:
                ctr_0+=1
        else:
            ctr_not+=1
    return ctr_3,ctr_2,ctr_1,ctr_0,ctr_not

In [21]:
test_abs=pd.read_pickle('test_abs')
n=len(test_abs)
ctr=0
avg_precision=[0]*11
valid_file=list(judge.keys())
op=[]
for i in range(n):
    if i in valid_file:        
        res=predict(test_abs['abstract'].iloc[i])
        #print(res)
        res1=gndtht_stats(res,judge,i)
        op.append([i,res1])

In [22]:
for entry in op:
    file_id=entry[0]
    stats=entry[1]
    label=['No of judgement with rel 3:','No of judgement with rel 2:','No of judgement with rel 1:','No of judgement with rel 0:','No of judgement not in ground thruth:']
    print('File Id:',file_id)
    for i in range(5):
        print(label[i],stats[i])
    print('######################################')

File Id: 0
No of judgement with rel 3: 2
No of judgement with rel 2: 0
No of judgement with rel 1: 3
No of judgement with rel 0: 0
No of judgement not in ground thruth: 5
######################################
File Id: 1
No of judgement with rel 3: 0
No of judgement with rel 2: 4
No of judgement with rel 1: 1
No of judgement with rel 0: 1
No of judgement not in ground thruth: 4
######################################
File Id: 2
No of judgement with rel 3: 0
No of judgement with rel 2: 1
No of judgement with rel 1: 2
No of judgement with rel 0: 2
No of judgement not in ground thruth: 5
######################################
File Id: 3
No of judgement with rel 3: 1
No of judgement with rel 2: 0
No of judgement with rel 1: 0
No of judgement with rel 0: 3
No of judgement not in ground thruth: 6
######################################
File Id: 4
No of judgement with rel 3: 0
No of judgement with rel 2: 2
No of judgement with rel 1: 0
No of judgement with rel 0: 1
No of judgement not in ground

## Language model

# Experimental

## Loading saved Lda model,dictionary,tfidf vectorizer

In [8]:
clean_text=list(word_tokenize(cleantext(text))) 
bow_text=dictionary.doc2bow(clean_text)
tfidf_text=tfidf[bow_text]
topic=lda_model[tfidf_text]

topic_vec=np.zeros(100)
for ind,wt in topic:
    topic_vec[ind]=wt

In [9]:
topic

[(10, 0.14324963),
 (20, 0.254431),
 (29, 0.15578026),
 (71, 0.041963633),
 (97, 0.24178416)]

## Topic of interest and community

In [10]:
no_top_topic=3
toi=[]#topic of interest
if len(topic)<no_top_topic:
    for ind,wt in topic:
        toi.append(ind)
else:
    topic.sort(key=lambda x:x[1],reverse=True)
    for i in range(no_top_topic):
        toi.append(topic[i][0])
        
    
toi_comm=[topic_comm[i] for i in toi]
toi_comm=set(toi_comm)
topic_rest=[]
for index,comm_no in enumerate(topic_comm):
    if comm_no in toi_comm and not index in toi:
        topic_rest.append(index)
        

In [11]:
toi_comm

{1, 2, 4}

In [12]:
len(topic_rest)

66

In [13]:
toi

[20, 97, 29]

## Paper Level

In [14]:
poi=[]#papers having maximuim content of toi
paper_vec_lt=list(paper_vec2.values())
for i in toi:
    samp=get_column_slice(paper_vec_lt,i)
    ind=np.argmax(np.array(samp))
    poi.append(ind)
puc=[]#papers under consideration
for i in topic_rest:
    topic_paper=get_column_slice(paper_vec_lt,i)
    n1=len(np.where(topic_paper>0)[0])
    thres=sum(topic_paper)/n1
    ind_list=np.where(topic_paper>thres)[0]
    for ele in ind_list:
        puc.append(ele)
puc=list(set(puc))

In [15]:
poi

[11053, 1885, 1568]

In [16]:
paper_scr=[]
for i in puc:
    num=bc_mat[i]+cocit_mat[i]
    den=min([dist[i][j] for j in poi])
    scr=num/den
    paper_scr.append((scr,i))

In [17]:
scr=[scr for scr,ind in paper_scr]

In [18]:
val=np.percentile(scr,90)

In [19]:
potential_paper=[]
for score,ind in paper_scr:
    if score>val:
        potential_paper.append(ind)

In [20]:
len(potential_paper)

684

## Reviewer Identification

In [21]:
roi=[]
ct_roi=[]
for i in poi:
    roi=roi+paper_auth[i]
rest_reviewer=[]
for i in potential_paper:
    if not i in roi:
        rest_reviewer=rest_reviewer+paper_auth[i]

In [22]:
roi

[233, 5, 231]

In [23]:
rest_reviewer=list(set(rest_reviewer))

In [24]:
len(rest_reviewer)

176

### Start Node selection

In [57]:
reviewer_detail=pd.read_pickle(path1+'auth_detail.pkl')

In [58]:
npaper=list(reviewer_detail['pc'])
citn=list(reviewer_detail['cn'])

In [59]:
dec_scr=[]
for i in roi:
    sim=1-cosine(reviewer_vec[i],topic_vec)
    if np.isnan(sim):
        sim=0
    val=0.15*npaper[i]/max(npaper)+0.15*citn[i]/max(citn)+0.7*sim
    dec_scr.append((val,i))
dec_scr.sort(reverse=True)
start_node=dec_scr[0][1]

In [60]:
start_node

233

## Random Walk with Restart

In [127]:
with open(path1+'reviewer_matrix.pkl','rb') as f:
    reviewer_matrix=pickle.load(f)

In [128]:
n=len(review_lt)
total_node=set([i for i in range(n)])
invalid_node=total_node-set(roi)-set(rest_reviewer)

In [129]:
custom_matrix=reviewer_matrix[:]

In [130]:
for node in invalid_node:
    for i in range(n):
        custom_matrix[node][i]=0
        custom_matrix[i][node]=0

In [131]:
col_sum=custom_matrix.sum(axis=1)
transition_prob=[]
for i in range(n):
    op=[]
    for j in range(n):
        if col_sum[j]:
            val=custom_matrix[i][j]/col_sum[j]
        else:
            val=0
        op.append(val)
    transition_prob.append(op)

In [132]:
val=[]
for j in range(n):
    temp=0
    for i in range(n):
        temp+=transition_prob[i][j]
    val.append(temp)

In [133]:
ini_state=[0]*n
ini_state[start_node]=1
len_op=10#No of items to be shown as output
new_state=ini_state
for i in range(10):
    new_state=np.dot(0.7,np.dot(transition_prob,new_state))+np.dot(0.3,ini_state)

In [134]:
np.count_nonzero(new_state)

176

In [135]:
op=new_state.argsort()[::-1][:30]

In [136]:
op_lt=[]
ctr=0
for i in op:
    if not i in invalid_node:
        op_lt.append(i)
        ctr+=1
    if ctr==10:
        break

In [137]:
op_lt

[233, 52, 22, 294, 227, 39, 100, 107, 91, 57]

In [332]:
samp_judge=judge[0]

In [333]:
samp_judge_review=[sno for sno,rat in samp_judge]

#### Evaluation

In [334]:
op_lt=predict(test_abs['abstract'].iloc[0])
for i in op_lt:    
    if not i in samp_judge_review:
        sim=float('-inf')
        rat=0
        for ind,j in enumerate(samp_judge_review):
            v1=1-cosine(reviewer_vec[i-1],reviewer_vec[j-1])
            if v1>sim:
                sim=v1
                rat=samp_judge[ind][1]
        samp_judge.append([i,rat])
        print([i,rat])
        print('YEAH')
        
        

In [355]:
def precision_k(res,gnd_tht,k,paper_no):
    actual=gnd_tht[paper_no]
    actual_auth=[i[0] for i in actual]
    rating=[i[1] for i in actual]
    n=len(actual)
    soft_p=0
    #print(actual)
    for i in range(k):
        pred=res[i]
        for j in range(n):
            if actual_auth[j]==pred:
                if rating[j]==3:
                    soft_p+=1
                elif rating[j]==2:
                    soft_p+=0.67
                elif rating[j]==1:
                    soft_p+=0.33
                break
    soft_p=soft_p/k
    return soft_p

In [336]:
avg_precision=[0]*11
#print(res)
for k in range(1,11):
    avg_precision[k]=precision_k1(res,samp_judge,k,i)
ctr=1
avg_precision=[ele/ctr for ele in avg_precision]

In [356]:
avg_precision=[0]*11
res=predict(test_abs['abstract'].iloc[0])
#print(res)
for k in range(1,11):
    avg_precision[k]=precision_k(res,judge,k,0)
ctr=1
avg_precision=[ele/ctr for ele in avg_precision]

In [357]:
avg_precision[1:]

[0.33,
 0.165,
 0.44333333333333336,
 0.3325,
 0.4,
 0.5,
 0.42857142857142855,
 0.41625,
 0.4444444444444444,
 0.4]

In [198]:
def precise_k(res,gnd_tht,k,paper_no):
    actual=gnd_tht[paper_no]
    actual_auth=[i[0] for i in actual]
    rating=[i[1] for i in actual]
    n=len(actual)
    soft_p=0
    t3e=0
    t2o=0
    o1e=0
    n9e=0
    z0r=0
    for i in range(k):
        pred=res[i]
        for j in range(n):
            if actual_auth[j]==pred:
                if rating[j]==3:
                    soft_p+=1
                    t3e+=1
                elif rating[j]==2:
                    soft_p+=0.67
                    t2o+=1
                elif rating[j]==1:
                    soft_p+=0.33
                    o1e+=1
                else:
                    z0r+=1
                    break
            n9e+=1
    soft_p=soft_p/k
    return soft_p


In [ ]:
test_abs=pd.read_pickle('test_abs')
n=len(test_abs)
ctr=0
avp=0
valid_file=list(judge.keys())
for i in range(n):
    if i in valid_file:        
        res=predict(test_abs['abstract'].iloc[i])
        #print(res)
        avp+=precise_k(res,judge,1,i)
        ctr+=1
print(avp/ctr)